# Computer Vision PS (WS21/22)


# Exercise sheet D (ExD)

**Group members**: Ilpo Viertola, Niklas Nurminen

**Total (possible) points**: 14

In [4]:
import torch
import torch.nn as nn
from torch import tensor
from torch.autograd import grad
import random
import numpy as np

## ExD.1 (3 points)

Let us assume we have a point $\mathbf{x} \in \mathbb{R}^3$ and 10 additional points $\mathbf{y}_1,\ldots,\mathbf{y}_{10}$ with $\forall i: \mathbf{y}_i \in \mathbb{R}^3$.

Now, say we want to compute (1) 

$$\mathbf{o}_{1i} = \mathbf{y}_i^2\enspace,$$

then (2)

$$ o_{2i} = \exp\left( \frac{-\| \mathbf{x} - \mathbf{o}_{1i} \|^2}{\sigma} \right)$$

followed by (3)

$$ o_{3} = \sum_{i} o_{2i}$$

and (4)

$$ o_4 = \frac{1}{1+\exp{(-o_3)}}$$

If we want to minimize $o_4$ over $\mathbf{y}_1,\ldots,\mathbf{y}_{10}$ via gradient descent, we need gradients wrt. those points, i.e.,

$$ \frac{\partial o_4}{\partial \mathbf{y}_i}\enspace.$$

In [22]:
# set random seed
torch.manual_seed(123)

y = torch.randn(10, 3, requires_grad=True)  # y_i's with respect to which we need gradients
x = tensor([8.,3.,4.])                      # x

Below is an implementation of (1)-(4) from above. Use `torch.autograd.grad` to compute gradients wrt. the $\mathbf{y}_i$.

In [23]:
o1 = torch.pow(y, 2.)
o2 = torch.exp(-(x-o1).norm(dim=1, p=2)**2)
o3 = o2.sum()
o4 = torch.sigmoid(o3)

Compute $\frac{\partial o_4}{\partial \mathbf{y}_i}$ and explain why we do not get any useful gradients.

In [24]:
g = grad(o4, y)

In [34]:
print(g[0])

tensor([[ 9.0438e-38, -1.7938e-38, -4.0320e-38],
        [-1.2428e-34,  2.7708e-35,  6.4985e-35],
        [-1.5273e-36, -9.4707e-37,  2.2886e-36],
        [-4.0044e-23,  8.9564e-24, -1.4484e-23],
        [ 2.9396e-30, -2.3059e-30,  5.9859e-30],
        [-4.2240e-20, -8.0011e-21,  3.4357e-21],
        [-5.2038e-13,  1.4091e-13, -1.0579e-13],
        [-1.4158e-28, -1.3859e-29, -2.7912e-29],
        [ 2.1651e-28,  3.0922e-29,  7.6932e-29],
        [-5.7877e-39,  1.2163e-39, -1.5445e-39]])


**Explanation**: The reason why we do not get any useful gradients is that the function we're trying to minimize is function of two variables ***x*** and ***y***. We need to compute the gradient in respect to both of the variables if we want to have any useful results.

## ExD.2 (5 points)

Say we have a linear map $f: \mathbb{R}^{100} \to \mathbb{R}^{45}, \ \mathbf{x} \mapsto \mathbf{A}\mathbf{x}$, implemented via `nn.Linear(100, 45, bias=False)` and we want to compute the *Lipschitz* constant of this linear map. 

**Definition**: A function $f: \mathbb{R}^n \to \mathbb{R}^m$ is called Lipschitz continuous if there exists a constant $L \geq 0$ such that 

$$\forall \mathbf{x},\mathbf{y} \in \mathbb{R}^n: \| f(\mathbf{x}) - f(\mathbf{y})\|_2 \leq L \| \mathbf{x}-\mathbf{y} \|_2$$

For a linear map (as above), the Lipschitz constant is given by the largest *singular value* of $\mathbf{A}$.

For this exercise, (1) initialize a linear layer with `nn.Linear(100, 45, bias=False)` and then (2) compute the Lipschitz constant (look into `torch.linalg` for help). Subsequently, write a function that takes, as arguments, a linear layer (i.e., a variable of type `nn.Linear`), a desired Lipschitz constant $L' \geq 0$ and reconfigures the weight matrix such that this Lipschitz constant is satisfied.

In [73]:
torch.manual_seed(1234)
# (1) initialize the linear layer in the configurations specified above
ll = nn.Linear(100, 45, bias=False)

# (2) compute Lipschitz constant (L)
U, S, Vh = torch.linalg.svd(ll.weight)
L = torch.max(S)

print('L(f): {:.3f}'.format(L))

L(f): 0.937


In [104]:
accuracy = 4
def constrain(f: nn.Linear, L:float):
    assert L >= 0
    L = np.round(L, accuracy)
    print("Target Lipschitz-constant is {:.3f}".format(L))

    U, S, Vh = torch.linalg.svd(f.weight)
    L_cur = torch.round(torch.max(S) * 10**accuracy) / (10**accuracy)

    if L == L_cur:
        print("The weight matrix already satisfies the Lipschitz-constant.")
        return
    
    elif L < L_cur:
        print("Current Lipschitz-constant {:.3f} is too large.".format(L_cur))
        ind = (S > L).nonzero(as_tuple=False)
        new_vals = [L]
        for _ in range(len(ind) - 1):
            new_vals.append(random.uniform((L - L*0.5), (L - L*0.01)))
        random.shuffle(new_vals)
        
        for i, val in enumerate(ind):
            S[val] = new_vals[i]
    
    else:
        print("Current Lipschitz-constant {:.3f} is too small.".format(L_cur))
        _, ind = torch.topk(S, 1)
        S[ind] = L

    # Reconstruct the weight matrix
    S_new = torch.zeros((f.weight.shape[0], f.weight.shape[1]))
    S_new[:f.weight.shape[0], :f.weight.shape[0]] = torch.diag(S)
    f.weight = torch.nn.Parameter(torch.matmul(U, torch.matmul(S_new, Vh)))

torch.manual_seed(1234)
f = nn.Linear(100, 45, bias=False)
L = random.random()
constrain(f, L)
# Test
constrain(f, L)

Target Lipschitz-constant is 0.894
Current Lipschitz-constant 0.937 is too large.
Target Lipschitz-constant is 0.894
The weight matrix already satisfies the Lipschitz-constant.


## ExD.3 (6 points)

Say you have a grayscale image of size $W \times H$, i.e., one ``color'' channel. This image can be stored as a $1 \times W \times H$ tensor. In case you have multiple ($B$) such images, you could store them as a $B \times 1 \times W \times H$ tensor.

In [33]:
# e.g., 6 (random) images of size 32x32
torch.manual_seed(1234)
img = torch.randn(6,1,32,32)

The idea of this exercise is, to first split the image into $P \times P$ patches (e.g., $8 \times 8$), then vectorize each patch and forward each vectorized patch through a simple MLP, implementing

$$f: \mathbf{x} \mapsto \mathbf{B}\text{ReLU}(\mathbf{A}\mathbf{x})$$

where $\mathbf{A} \in \mathbb{R}^{P^2 \times P^2}$ and $\mathbf{B} \in \mathbb{R}^{P^2 \times 128}$. 

**Evaluation**: use the `img` tensor from above as input, and take $P=8$. The output should then be a tensor of size $6 \times 16 \times 128$.




In [48]:
# imports that require einops
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import torch.nn.functional as F
from sympy import symbols, solve, Eq

In [105]:
patch_H, patch_W = 8, 8
patch_D = patch_W * patch_H
embedding_dim = 128

mapping = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', 
                       p1 = patch_H, 
                       p2 = patch_W),
            nn.Linear(patch_D, patch_D, bias=False),
            nn.ReLU(),
            nn.Linear(patch_D, embedding_dim, bias=False)
        )

In [106]:
import numpy as np
np.sum([p.numel() for p in mapping.parameters()])

12288

In [107]:
# test with
print(img.size())
out_1 = mapping(img)
print(out.size())

torch.Size([6, 1, 32, 32])
torch.Size([6, 16, 64])


In [111]:
# Below is a template class that you can use 

class MyOp(nn.Module):
    def __init__(self, patch_W=8, patch_H=8, embedding_dim=128):
        super().__init__()
        
        assert patch_W == patch_H
        
        self.patch_W = patch_W
        self.patch_H = patch_H
        self.embedding_dim = embedding_dim
        self.patch_D = patch_H * patch_W
        
        # self.layers = nn.Sequential(
        #     nn.Conv2d(1, self.patch_H * 2, kernel_size=25, bias=False),
        #     nn.Flatten(2,-1),
        #     nn.Linear(self.patch_D, self.patch_D, bias=False),
        #     nn.ReLU(),
        #     nn.Linear(patch_D, self.embedding_dim, bias=False)
        # )
        self.l1 = nn.Conv2d(1, self.patch_H * 2, kernel_size=25, bias=False)
        self.l2 = nn.Flatten(2,-1)
        self.l3 = nn.Linear(self.patch_D, self.patch_D, bias=False)
        self.l4 = nn.ReLU()
        self.l5 = nn.Linear(patch_D, self.embedding_dim, bias=False)
    
    def forward(self, x_0):
        c = self.calculate_channel_amnt(x_0)
        K, S = self.calculate_kernel_stride_size(x_0)
        self.l1 = nn.Conv2d(in_channels=1, out_channels=c, kernel_size=K, stride=S, bias=False)

        print("Before the network:", x_0.size())
        x_1 = self.l1(x_0)
        print("After Conv2d:", x_1.size())
        x_2 = self.l2(x_1)
        print("After Flatten:", x_2.size())
        x_3 = self.l3(x_2)
        print("After 1st Linear:", x_3.size())
        x_4 = self.l4(x_3)
        print("After ReLU:", x_4.size())
        x_5 = self.l5(x_4)
        print("After final layer (Linear):", x_5.size())
        return x_5
    
    def calculate_channel_amnt(self, x: torch.Tensor):
        H = x.shape[-2]
        W = x.shape[-1]
        
        # w * self.patch_W = W
        # h * self.patch_H = H
        # c = h * w => 
        return int((H / self.patch_H) * (W / self.patch_W))
    
    def calculate_kernel_stride_size(self, x: torch.Tensor):
        # out_size = P x P = self.patch_W x self.patch_H
        H_in = x.shape[-2]
        W_in = x.shape[-1]
        # K = kernel_size, S = stride_length
        # self.patch_H = (H_in - 1 * (K - 1) - 1) / (S) + 1
        # self.patch_W = (W_in - 1 * (K - 1) - 1) / (S) + 1
        K, S = symbols('K, S')
        eq1 = Eq((H_in - 1 * (K - 1) - 1) / (S) + 1, self.patch_H)
        eq2 = Eq((W_in - 1 * (K - 1) - 1) / (S) + 1, self.patch_W)
        results = solve((eq1, eq2), (K,S))

        if len(results) == 1:  # K is expressed with S
            S = 1
            K = eval(str(results[K]).replace('S', str(S)))
            return K, S
        else:
            return results.get(K), results.get(S)


In [112]:
# test with
print("Image-tensor size:", img.size())
obj = MyOp(8,8,128)
out_2 = obj(img)
print("Are sizes after two networks the same?", out_2.size() == out_1.size())

Image-tensor size: torch.Size([6, 1, 32, 32])
Before the network: torch.Size([6, 1, 32, 32])
After Conv2d: torch.Size([6, 16, 8, 8])
After Flatten: torch.Size([6, 16, 64])
After 1st Linear: torch.Size([6, 16, 64])
After ReLU: torch.Size([6, 16, 64])
After final layer (Linear): torch.Size([6, 16, 128])
Are sizes after two networks the same? True
